In [1]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
import random

In [2]:
# id_df = pd.read_csv('Restauant-CusineUpdated_-_results-20230110-115819.csv')
# id_df = id_df[~id_df['short_name'].str.contains("DNU")]
# id_df = id_df[~id_df['short_name'].str.contains("dnu")]
# id_df.to_csv("All_Restauants.csv", index=False)
# print(id_df.shape)

In [3]:
# Reading info for customer and restaraunt profiles 
customer_profiles = pd.read_csv('./derived_files/customer_profiles_recency_all.csv')
restauraunt_profiles = pd.read_csv('./derived_files/combined_restaurant_order_profile_all.csv')
id_df = pd.read_csv('./derived_files/All_Restauants.csv')

print(customer_profiles.shape)
print(restauraunt_profiles.shape)

# Reading info for orders and restaraunt names, for later use
orders = pd.read_csv('./from_skip_files/orders_11mil.csv')
# restaraunt_names = pd.read_csv('restaurant_id_and_names.csv')
restaraunt_scores = pd.read_csv('./from_skip_files/Restaurant_Cuisine(LowerCase).csv')


(554785, 55)
(2354, 55)


In [4]:
id_df['short_name'] = id_df['short_name'].str.lower()
restauraunt_profiles = restauraunt_profiles[restauraunt_profiles['short_name'].isin(id_df['short_name'])]
print(restauraunt_profiles.shape)

(2250, 55)


##### Over here was try 4 different metrics to find similarity between vectors, Cosine Similarity, Square data transformation then Cosine Similarity, Euclidean Distance, Spearman Correlation Coefficient. It seems Cosine Similarity is a strong choice in terms of speed and results, with square root cosine being an interesting contender

In [65]:
def recommend_cosine_sim(customer_vector, restaraunt_profiles, num_recommendations):
    # Calculate cosine similarity between customer and restaraunts, and stores the indicies of the highest similarity
    # restaurants in variable value. 
    customer_cuisines = customer_vector.iloc[-54:].to_numpy()
    customer_cuisines = customer_cuisines.reshape(1, -1)
    restaurant_cuisines = restauraunt_profiles.iloc[:, 1:].to_numpy()
    similarity = cosine_similarity(customer_cuisines, restaurant_cuisines)
    value = np.argsort(similarity.flatten())[::-1]
    top_X_restaraunts_id = []
    top_X_short_names = []
    skip_scores = []
    # Ensures unique restaraunt names in recommendation
    for i in value:
        curr_name = restauraunt_profiles.iloc[i]['short_name']
        skip_score_same_name = id_df[id_df['short_name'] == curr_name]
        max_skip_score_row = skip_score_same_name.loc[skip_score_same_name['skip_score'].idxmax()]
        top_X_restaraunts_id.append(max_skip_score_row['restaurant_id'])
        skip_scores.append(max_skip_score_row['skip_score'])
#         top_X_restaraunts
        if len(top_X_restaraunts_id) == num_recommendations:
            break
    top_X_restaraunts = restauraunt_profiles.iloc[value[:num_recommendations]]
    top_cos_scores = np.sort(similarity.flatten())[::-1][:num_recommendations]
    top_X_restaraunts.insert(55,'cosine similarity score', top_cos_scores)
    top_X_restaraunts.insert(56,'skip score', skip_scores)
    top_X_restaraunts.insert(1,'restaraunt_id', top_X_restaraunts_id)
    return top_X_restaraunts

In [6]:
def pretty_print_recommendation(recommendation, customer_vector):
    c_row = customer_vector
    c_row = c_row.drop('customer_id')
    c_row = pd.to_numeric(c_row, errors='raise')
    c_top = c_row.nlargest(55)
    c_top.name = "Customer Preference"
    customer_orders = orders[orders['customer_id'] == customer_vector['customer_id']]
    order_restauraunt = customer_orders['restaurant_short_name'].to_list()
    order_item_list = customer_orders['item_list'].str.split(", ").to_list()
    for i in range(0, len(order_item_list)):
        print(order_restauraunt[i])
        print(order_item_list[i])
    for i in range(5):
        r_row = recommendation.iloc[i]
        restaraunt_name = r_row['short_name']
        restaraunt_score = id_df[id_df['restaurant_id'] == r_row['restaraunt_id']]['skip_score']
        r_row = r_row.drop('restaraunt_id')
        r_row = r_row.drop('short_name')
        r_row = pd.to_numeric(r_row, errors='raise')
        r_top = r_row.nlargest(56)
        r_top.name = "Restauraunt Offers"
        compare = pd.concat([c_top, r_top], axis=1)
        print(restaraunt_name)
        print(compare[:5])
        print("-------------")

##### Here is where we try the multiple different methods and compare results 

##### Here we add variety into our system by picking the top 2 cuisines a customer prefers and replacing them with a similar cuisine. The similar cuisines were calculated using a similarity matrix. We randomly pick one of the top 3 most similar cuisines as the replacement for the current cuisine. E.g Customer likes Fries, the top 3 similar cuisine are Pork, Soup, Coffee/Tea so we select one of those

In [14]:
rf = pd.read_csv("./derived_files/similarity_matrix.csv")
similar_cuisines = {}
similarity_upper_limit = 0.75
for index, row in rf.iterrows():
    similarities = row.drop("Unnamed: 0")
    similarities = pd.to_numeric(similarities, errors='raise')
#     columns = test.nlargest(8).tail(5)
#     Thresh hold of 0.78
    print(row['Unnamed: 0'], "\n",  similarities.nlargest(6))
    filtered = similarities[similarities <= similarity_upper_limit]
    print(row['Unnamed: 0'], "\n",  filtered.nlargest(6))
#     columns = columns.drop(row.iloc[0])
    columns = filtered.nlargest(5)
    similar_cuisines[row.iloc[0]] = columns

African 
 African              0.998775
Mexican              0.413728
Healthy              0.407949
Rice                 0.407633
Barbecue             0.407307
Sandwiches & Subs    0.406959
Name: 0, dtype: float64
African 
 Mexican              0.413728
Healthy              0.407949
Rice                 0.407633
Barbecue             0.407307
Sandwiches & Subs    0.406959
Indian               0.406314
Name: 0, dtype: float64
Alcohol 
 Alcohol      0.998775
Canadian     0.503802
Burgers      0.462023
Chicken      0.456689
Fast food    0.453504
Butcher      0.452890
Name: 1, dtype: float64
Alcohol 
 Canadian      0.503802
Burgers       0.462023
Chicken       0.456689
Fast food     0.453504
Butcher       0.452890
Coffee/Tea    0.447713
Name: 1, dtype: float64
Bakery 
 Bakery                 0.998775
Breakfast & Brunch     0.809318
Chicken                0.804058
Fast food              0.775131
Desserts               0.769674
Hot Dogs & Sausages    0.748637
Name: 2, dtype: float64
Bakery 
 

In [8]:
def update_3_top_cuisines(customer_vector, similarity_matrix):
    # Picks two highest customer cuisine preferences and swaps them with similar cuisines found 
    # in a similarity matrix. E.g customer likes Chicken 0.25 and Rice 0.2, will swap them out so they 
    # could become Fast Food and Lamb.
    customer_vector_id = customer_vector.iloc[0]
    customer_cuisines =  customer_vector.iloc[-54:]
    org_sum = str(customer_cuisines.sum())
    top3 =  pd.to_numeric(customer_cuisines, errors='raise').nlargest(3)
    print(pd.to_numeric(customer_cuisines, errors='raise').nlargest(5))
    do_not_repeat = top3.index.to_list()
    print(do_not_repeat)
    for cuisine in top3.index.to_list():
        similar_cuisines_dict = similarity_matrix[cuisine].to_dict()
        print(similar_cuisines_dict)
        old_cuisine = cuisine
        old_cuisine_preference = top3[old_cuisine]
        print(old_cuisine, old_cuisine_preference)
        similar_cuisine = ""
        while similar_cuisine == "" or similar_cuisine in do_not_repeat:
            similar_cuisine = random.choice(list(similar_cuisines_dict.keys()))
        similarity_of_new_cuisine = similar_cuisines_dict[similar_cuisine]
        print(similar_cuisine, similarity_of_new_cuisine)
        updated_old_cuisine_value = customer_cuisines[similar_cuisine] 
        updated_similar_cuisine_value = old_cuisine_preference
        customer_cuisines[old_cuisine] = updated_old_cuisine_value
        customer_cuisines[similar_cuisine] = updated_similar_cuisine_value
    print("Sums", org_sum, " ", customer_cuisines.sum())
    top5 = pd.to_numeric(customer_cuisines, errors='raise').nlargest(5)
    print(top5)
    customer_id_series = pd.Series({"customer_id": customer_vector_id})
    final_test_customer = pd.concat([customer_id_series, customer_cuisines])
    return final_test_customer

##### Now we try to include the skip score as part of the equation when recommending restaruants. The new formula for calculating top restauraunts is <b>score = (weight)*(skip_score)+(1-weight)*(cosine_similarity)</b> where weight is a user set variable. In our case we set weight to 0.2

In [58]:
def adjust_recommendations_using_skip_score(top_restaraunts, weight):
    if weight < 0 or weight > 1:
        return
    adjusted_scores = []
    for i in range(0, top_restaraunts.shape[0]):
        r_row = top_restaraunts.iloc[i]
#         restaraunt_score = id_df[id_df['restaurant_id'] == r_row['restaraunt_id']]['skip_score']
#         print(restaraunt_score.iloc[0], " ==== ", r_row['skip score'])
        adjusted_scores.append(weight*r_row['skip score']/100+(1-weight)*
                               r_row['cosine similarity score'])
    top_restaraunts.insert(57,'adjusted score', adjusted_scores)
#     print(top_restaraunts[['cosine similarity score', 'adjusted score']])
    new_top_restaraunts = top_restaraunts.sort_values(by='adjusted score', ascending=False)
    return new_top_restaraunts

In [10]:
new_top_restaraunts = adjust_recommendations_using_skip_score(top_restaraunts, 0.1)

NameError: name 'top_restaraunts' is not defined

In [80]:
def diff_lists(list1, list2):
    return set(list1)-set(list2)

##### Trying to combine all the steps above into one solution, find top 5 restaraunts using cosine sim then apply skip score weighing. Replace bottom two restaraunts with varied restaraunts using similarity matrix for similar foods (also apply skip score weighting to find top two restaraunts) then finally recommend top 5

In [105]:
def recommend_pipeline():
    avoid_restaraunts = []
    random_number = random.randint(0, 100000) 
    top_restaraunts = recommend_cosine_sim(customer_profiles.iloc[random_number], restauraunt_profiles, 5)
    new_customer_vector =  update_3_top_cuisines(customer_profiles.iloc[random_number], similar_cuisines)
    top_restaraunts_varied = recommend_cosine_sim(new_customer_vector, restauraunt_profiles, 5)
    top_restaraunts_adjusted = adjust_recommendations_using_skip_score(top_restaraunts, 0.1)
    top_restaraunts_varied_adjusted = adjust_recommendations_using_skip_score(top_restaraunts_varied, 0.1)
    print(top_restaraunts_adjusted[['cosine similarity score', 'adjusted score']])
    print(top_restaraunts_varied_adjusted[['cosine similarity score', 'adjusted score']])
    diff = diff_lists(top_restaraunts_varied_adjusted['short_name'].to_list(), top_restaraunts['short_name'].to_list())
    print(len(diff))
    top_restaraunts_varied_adjusted = top_restaraunts_varied_adjusted[top_restaraunts_varied_adjusted['short_name'].isin(diff)]
    intermediate_top_3 = top_restaraunts_adjusted.head(3)
    intermediate_top_2 = top_restaraunts_varied_adjusted.head(2)
    final_recommendations = pd.concat([intermediate_top_3, intermediate_top_2])
#     print(final_recommendations)
    pretty_print_recommendation(final_recommendations, customer_profiles.iloc[random_number])
    return final_recommendations

In [106]:
recommendations = recommend_pipeline()
# Error that I got to fix, no repeats in top 3/bottom 2

Fast food             0.151281
Pizza                 0.137032
Breakfast & Brunch    0.108141
Pork                  0.093639
Beef                  0.091311
Name: 98814, dtype: float64
['Fast food', 'Pizza', 'Breakfast & Brunch']
{'Pork': 0.7240193813213287, 'Fries': 0.710510482362637, 'Breakfast & Brunch': 0.6911385448002773, 'Mediterranean': 0.6783670856298296, 'Hot Dogs & Sausages': 0.6702255724721979}
Fast food 0.1512814340209622
Fries 0.710510482362637
{'Chicken': 0.6326665297158154, 'Pub food': 0.6283980329898301, 'Healthy': 0.569619470244107, 'Barbecue': 0.558832735675784, 'Vegetarian': 0.5533299882985628}
Pizza 0.1370319617791933
Healthy 0.569619470244107
{'Sandwiches & Subs': 0.7116300357505128, 'Fast food': 0.6911385448002773, 'Pork': 0.6889306298615304, 'Coffee/Tea': 0.6648813651544385, 'Chicken': 0.6532789657495197}
Breakfast & Brunch 0.1081410565040801
Sandwiches & Subs 0.7116300357505128
Sums 0.9999999999999994   0.9999999999999994
Fries                0.151281
Healthy     